# EMBS Training CNN Demo
*Adapted from PyTorch Docs*

## Basic Imports

TODO - Import the following:
- `torch`: Main pytorch package
- `torchvision`: Computer vision pytorch package

In [ ]:
### TODO - Import torch & torchvision ###

## Normalizing our data

We use a transformer to help us ensure consistency with our images through normalization

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

## Loading our data
We create the following variables to store outputs for training and testing data sets:
- `trainset`: stores the training dataset
- `testset`: stores the test dataset
- `trainloader`: stores a reference to the loaded training dataset
- `testloader`: stores a reference to the loaded training dataset

TODO - Complete the following:
- store the CIFAR10 *test* dataset in `testset` (Hint: very similar to storing trainset but set the train parameter to false)
- load the test set into trainloader (Again, similar to its training counter-part, no need to shuffle though)
- Create a tuple called `classes` with the following class labels (strings):
  ```
  'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
  ```

In [ ]:
batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = """TODO"""

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)
testloader = """TODO"""

classes = """TODO"""

## Printing the Images

Let's validate that these have been loaded properly

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

## The Neural Network

TODO:
Copy-paste your NN from the previous walk-through and modify it to take 3-channel images (R, G, B) instead of 1-channel input. (Change the first input of our first layer from 1 to 3)

In [ ]:
### TODO: copy-pase your NN and modify it to accept 3-channel images ###

net = Net() # Instantiating your network. Change Net() to <your-network-name>()

### Using a loss function & optimzer

These are used to assess how good or bad our network is doing and update our weights accordingly.

TODOS:
- set `criterion` to `CrossEntorypyLoss` (found in `nn`)
- set `optimizer` to Stochastic Gradient Descent with Momentum or `SGD` (found in `optim`) with parameters `(net.parameters(), lr=0.001, momentum=0.9)` 

In [ ]:
import torch.optim as optim

criterion = """TODO"""
optimizer = """TODO"""

### Training our network

Now, we iterate over our dataset for *n* number of "epochs" or iterations to update the weights of our network so it can "learn".

TODO
1. set `num_epochs` to `5` to control the number of epochs (5 in our case)
2. extract inputs and labels from `data` (`data` is a list of the form `[inputs, labels]`)
3. set `outputs` equal to the result of passing `inputs` into `net()`
4. back-propagate loss (hint: `loss` has a function called `backward()`)
5. step stochastic gradient descent aka our optimizer (hint: `optimizer` has a function called `step()`)

In [ ]:
num_epochs = """TODO 1"""
for epoch in range(num_epochs): # iterate over the data-set num_epochs times

    running_loss = 0.0 # keep track of loss for the given epoch
    for i, data in enumerate(trainloader, 0):
        inputs = """TODO 2"""
        labels = """TODO 2"""

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = """TODO 3"""
        loss = criterion(outputs, labels)
        """TODO 4"""
        """TODO 5"""

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

### Save our model

Saving it to a file called `cifar_net.pth`

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

### Loading our Testing Data

TODO:
- Create an iterator from our testloader (use the `iter()` function and pass in the `testloader` variable we created earlier)

In [ ]:
dataiter = """TODO"""
images, labels = next(dataiter)

# print images and labels
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

### Testing our model

We first load our saved model and then pass in our test images. Then we extract the most likely class from the list of probablities of each class

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH)) # load our saved model

outputs = net(images) # pass in the images to our neural network

probablities, predicted = torch.max(outputs, 1) # extract the most likely class

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

### Overall Accuracy

Lets run through each of our test images and see how our model performs

In [ ]:
correct = 0
total = 0
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest probability is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # tally up the correct predictions for each class
        for label, prediction in zip(labels, predicted):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')